In [1]:
!pip3 install numpy==1.15.4

     |████████████████████████████████| 13.9MB 238kB/s 
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.15.4 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.0.1.post1 has requirement numpy>=1.16, but you'll have numpy 1.15.4 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [1]:
import pandas as pd
import numpy as np
import re
import json
import pickle

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR

from tqdm import tqdm
from tqdm import tqdm_notebook
from tqdm import notebook

from sklearn.model_selection import train_test_split

import h5py
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [3]:
cd drive/My\ Drive

/content/drive/My Drive


In [4]:
ls

 CNN_character.ipynb         embedding_matrix.h5       model
 CNN_word_embedding.ipynb    embedding_matrix.tar.gz   name_category.h5
'Colab Notebooks'/           embeddings_keywords.h5    word_nparray.h5
 data_conv_experimental.h5   keywords.pkl              word_nparray.tar.gz


In [5]:
def labels_map(y):
    S=set(y)
    num=len(S)
    dic={word:i for i, word in enumerate(S)}
    
    labels=y.apply(lambda val: dic[val])
    
    return labels.values    

### all names and categories

In [6]:
df_all=pd.read_hdf('name_category.h5',key='data',mode='r')

In [ ]:
num_classes=len(set(df_all['category']))

In [ ]:
df_all['category'].value_counts()[:10]

Adviesbureaus - organisatieadvies en management    86212
Bouwbedrijf                                        45854
Diversen                                           29561
Computersoftware                                   24150
Kapper                                             22471
Adviesbureaus                                      19873
Schoonheidsspecialiste                             19279
Adviesbureaus - automatisering                     19058
Administratiekantoor                               16944
Reclame                                            16448
Name: category, dtype: int64

### embedding matrix and sequences

In [7]:
f=h5py.File('embedding_matrix.h5','r')
embedding_matrix=f['embedding_matrix']

g=h5py.File('word_nparray.h5','r')
word_nparray=g['word_nparray']
word_nparray=word_nparray[()]

### small data-set original categories

In [8]:
lst=['Bouwbedrijf','Computersoftware','Kapper','Reclame','Administratiekantoor','Adviesbureaus']

In [9]:
df_reduced=df_all[df_all['category'].isin(lst)]
idx=df_all['category'].isin(lst)

In [10]:
word_reduced=word_nparray[idx.values,:]

In [11]:
num_classes=len(set(df_reduced['category']))
labels=labels_map(df_reduced['category'])

In [ ]:
num_classes=len(set(df_all['category']))
labels=labels_map(df_all['category'])

In [12]:
num_classes

6

In [29]:
len(word_reduced)  

145740

## General categories

In [33]:
df=pd.read_hdf('data_conv_experimental.h5',key='data',mode='r')
df['category0'].value_counts()

opleiding          355211
diensten           262805
winkel             259861
vrije tijd         140906
gezondheidszorg    136865
gereedschap        127734
regering            99355
reizen              25868
boodschappen         3901
Name: category0, dtype: int64

In [34]:
lst=['diensten','winkel','gereedschap']
idx=  df['category0'].isin(lst)
df_reduced=df[idx].copy()
word_reduced=word_nparray[idx.values,:]
num_classes=len(set(df_reduced['category0']))
labels=labels_map(df_reduced['category0'])

In [35]:
num_classes

3

### Sequence lengths for the LSTM model

In [ ]:
name_len=[(word!=0).sum() for word in word_nparray]
name_len=np.array(name_len)

# Models: Conv1D, LSTM, FNN

In [51]:
class Conv1D(nn.Module):
    def __init__(self,embedding_data,max_length,output_dim):
        super(Conv1D,self).__init__()
        
        self.embedding_dim=embedding_data.shape[1]
        self.embed=nn.Embedding.from_pretrained(torch.from_numpy(embedding_data).float())
        self.conv1=nn.Sequential(nn.Conv1d(self.embedding_dim,300,kernel_size=1,padding=0),nn.BatchNorm1d(300),nn.ReLU(),nn.MaxPool1d(4))
        self.conv2=nn.Sequential(nn.Conv1d(200,200,kernel_size=2,padding=0),nn.BatchNorm1d(200),nn.ReLU(),nn.MaxPool1d(2))
        self.conv3=nn.Sequential(nn.Conv1d(200,200,kernel_size=1,padding=0),nn.ReLU())
        self.max_length=max_length

        input_shape=(128,self.max_length,self.embedding_dim)
        self.output_conv_dim = self.get_conv_output(input_shape)
        
        self.fc1 = nn.Sequential(nn.Linear(self.output_conv_dim, 500), nn.ReLU()  )

        self.fc2 = nn.Sequential( nn.Linear(500, 500), nn.ReLU(),nn.Dropout(0.3) )
        
        self.fc3=nn.Linear(500,output_dim)
        
    def get_conv_output(self,shape):
        x = torch.rand(shape)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        #x = self.conv2(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        out_dim = x.size(1)
        return out_dim
        
        
        
    def forward(self,x):
        x=x[:,:self.max_length]
        x=self.embed(x)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        #x = self.conv2(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [ ]:
class Conv2D(nn.Module):
    def __init__(self,embedding_data,max_length,output_dim):
        super(Conv2D,self).__init__()
        
        self.embedding_dim=embedding_data.shape[1]
        self.embed=nn.Embedding.from_pretrained(torch.from_numpy(embedding_data).float())
        self.conv1=nn.Sequential(nn.Conv2d(1,200,kernel_size=1,padding=0),nn.ReLU(),nn.MaxPool2d(2))
        self.conv2=nn.Sequential(nn.Conv2d(200,200,kernel_size=2,padding=0),nn.ReLU(),nn.MaxPool2d(2))
        self.conv3=nn.Sequential(nn.Conv1d(200,200,kernel_size=1,padding=0),nn.ReLU())
        self.max_length=max_length

        input_shape=(128,1,self.max_length,self.embedding_dim)
        self.output_conv_dim = self.get_conv_output(input_shape)
        
        self.fc1 = nn.Sequential(nn.Linear(self.output_conv_dim, 300), nn.ReLU()  )

        self.fc2 = nn.Sequential( nn.Linear(300, 300), nn.ReLU(),nn.Dropout(0.5) )
        
        self.fc3=nn.Linear(300,output_dim)
        
    def get_conv_output(self,shape):
        x = torch.rand(shape)
        x = x.transpose(2, 3)
        x = self.conv1(x)
        x = self.conv2(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        out_dim = x.size(1)
        return out_dim
        
        
        
    def forward(self,x):
        x=x[:,:self.max_length]
        x=self.embed(x)
        x=x.view(x.size(0),1,x.size(1),x.size(2))
        x = x.transpose(2, 3)
        x = self.conv1(x)
        x = self.conv2(x)
        #x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [ ]:
class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_data,hidden_dim, num_classes):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding_dim=embedding_data.shape[1]
        self.embed=nn.Embedding.from_pretrained(torch.from_numpy(embedding_data).float())
        self.lstm = nn.LSTM(self.embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=num_classes)

    def forward(self, x,lens):
        """
        Perform a forward pass of our model on some input.
        """
        embeds = self.embed(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[range(len(lens)),lens-1]
        return out.squeeze()

#### number of model parameters

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

# Training scripts:

In [14]:
def train(model, train_loader, epochs, criterion, optimizer, device):
    L=len(train_loader)
    var_losses=[]
    for epoch in range(1, epochs + 1):
        model.train() # Make sure that the model is in training mode.

        total_loss = 0

        for batch in notebook.tqdm(train_loader):
            # get data
            batch_x, batch_y = batch
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()

            # get predictions from model
            y_pred = model(batch_x)
        
            # perform backprop
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        
        print("Epoch: {}, Loss: {}".format(epoch, total_loss / L))

### LSTM training script differs a bit

In [ ]:
def train_lstm(model, train_loader, epochs, criterion, optimizer, device):
    L=len(train_loader)
    var_losses=[]
    for epoch in range(1, epochs + 1):
        model.train() # Make sure that the model is in training mode.

        total_loss = 0

        for batch in notebook.tqdm(train_loader):
            # get data
            batch_x, batch_len, batch_y = batch
            
            batch_x = batch_x.to(device)
            batch_len=batch_len.to(device)
            batch_y = batch_y.to(device)

            optimizer.zero_grad()

            # get predictions from model
            y_pred = model(batch_x,batch_len)
        
            # perform backprop
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        
        print("Epoch: {}, Loss: {}".format(epoch, total_loss / L))

# Train/Test datasets preparation

In [36]:
indices=np.arange(word_reduced.shape[0])
np.random.shuffle(indices)
data_x=word_reduced[indices]
data_y=labels[indices]
#data_len=name_len[indices]

nb_validation=int(0.2*word_reduced.shape[0])

x_train=data_x[:-nb_validation]
#x_train_len=data_len[:-nb_validation]
y_train=data_y[:-nb_validation]

x_test=data_x[-nb_validation:]
#x_test_len=data_len[-nb_validation:]
y_test=data_y[-nb_validation:]

### Torch datasets and dataloaders

In [37]:
#CONV1D

x_train_torch=torch.from_numpy(x_train)
y_train_torch=torch.from_numpy(y_train)

train_ds = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128)

x_test_torch=torch.from_numpy(x_test)
y_test_torch=torch.from_numpy(y_test)

test_ds = torch.utils.data.TensorDataset(x_test_torch, y_test_torch)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128)

In [ ]:
#LSTM
x_train_torch=torch.from_numpy(x_train)
y_train_torch=torch.from_numpy(y_train)
x_train_len_torch=torch.from_numpy(x_train_len)

train_ds = torch.utils.data.TensorDataset(x_train_torch, x_train_len_torch, y_train_torch)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=512)

x_test_torch=torch.from_numpy(x_test)
y_test_torch=torch.from_numpy(y_test)
x_test_len_torch=torch.from_numpy(x_test_len)
test_ds = torch.utils.data.TensorDataset(x_test_torch, x_test_len_torch, y_test_torch)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=512)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device {}.".format(device))

Using device cuda.


In [52]:
model=Conv1D(embedding_matrix[()],max_length=10,output_dim=num_classes).to(device)

In [ ]:
model=Conv2D(embedding_matrix[()],max_length=6,output_dim=num_classes).to(device)

In [ ]:
model_lstm=LSTMClassifier(embedding_matrix[()],300,num_classes).to(device)

In [53]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

### Scheduler lr

In [ ]:
def lambda1(epoch):
  if epoch>=80 and epoch<120:
    return 0.1
  elif epoch>=120:
    return 0.1**2
  else:
    return 1
scheduler = LambdaLR(optimizer, lr_lambda=lambda1)

### training Conv1D

In [54]:
train(model, train_dl, 20, loss_fn,optimizer, device)


Epoch: 1, Loss: 0.7733604518836421



Epoch: 2, Loss: 0.730094779505501



Epoch: 3, Loss: 0.7125648609679857



Epoch: 4, Loss: 0.6992061899523601



Epoch: 5, Loss: 0.687253697463947


...


Epoch: 16, Loss: 0.584482957347586



Epoch: 17, Loss: 0.5781913839876285



Epoch: 18, Loss: 0.5701714282912669



Epoch: 19, Loss: 0.5646086980056059



Epoch: 20, Loss: 0.5572088004184972


## Accuracy and Confusion matrix

In [21]:
 def accuracy(model,val_dl):
  model.eval()
  with torch.no_grad():
        accu=0
        for batch in val_dl:
            val_X,val_y=batch
            val_X=val_X.to(device)
            val_y=val_y.to(device)
            x=model(val_X)
            probs=F.softmax(x,dim=1)
            preds=probs.argmax(dim=1)
            accu+=(preds==val_y).sum().item()
    
  accu=accu/len(val_dl.dataset)*100
  return accu

def accuracy_lstm(model,val_dl):
  model.eval()
  with torch.no_grad():
        accu=0
        for batch in val_dl:
            val_X,val_len, val_y=batch
            val_X=val_X.to(device)
            val_y=val_y.to(device)
            val_len=val_len.to(device)

            x=model(val_X,val_len)
            probs=F.softmax(x,dim=1)
            preds=probs.argmax(dim=1)
            accu+=(preds==val_y).sum().item()
    
  accu=accu/len(val_dl.dataset)*100
  return accu
  
def confusion_matrix(model,val_dl,nb_classes):
  conf_matrix = torch.zeros(nb_classes, nb_classes)
  model.eval()
  with torch.no_grad():
      for i, (inputs, classes) in enumerate(val_dl):
          inputs = inputs.to(device)
          classes = classes.to(device)
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          for t, p in zip(classes.view(-1), preds.view(-1)):
                  conf_matrix[t.long(), p.long()] += 1
  conf_sum=conf_matrix.sum(1)
  conf_sum=conf_sum.reshape(-1,1)
  print(conf_matrix/conf_sum*100)

def probs(model,val_dl):
  model.eval()
  prbs=torch.FloatTensor([]).to(device)
  with torch.no_grad():
        for batch in val_dl:
            val_X,val_y=batch
            val_X=val_X.to(device)
            val_y=val_y.to(device)
            x=model(val_X)
            probs=F.softmax(x,dim=1)
            preds=probs.argmax(dim=1)
            idx=(preds==val_y)
            preds=probs.max(dim=1)
            preds=preds.values[idx]
            prbs=torch.cat((prbs,preds))
    
  return prbs

In [22]:
len(train_ds)+len(test_ds)

145740

In [ ]:
accuracy_lstm(model_lstm,test_dl)

31.364991263233634

In [49]:
accuracy(model,train_dl)

75.2460024600246

In [50]:
accuracy(model,test_dl)

63.76768142681427

In [67]:
probabilities=probs(model,test_dl)

In [90]:
(probabilities>0.8).sum().item()/probabilities.size(0)

0.5015764158976366

In [ ]:
confusion_matrix(model,test_dl,num_classes)

tensor([[85.9753,  2.9943,  2.2017,  2.6420,  0.5724,  5.6143],
        [ 1.2508, 79.2407,  5.4861,  2.7650,  1.8653,  9.3921],
        [ 1.9727, 10.3722, 54.2201, 13.3415,  3.0710, 17.0226],
        [ 2.4788,  9.1596, 19.4075, 46.4027,  2.6904, 19.8609],
        [ 0.8753,  7.9384,  6.1274,  3.2599, 69.3027, 12.4962],
        [ 3.7685, 15.1927, 12.8331,  9.8455,  5.5678, 52.7924]])


### training LSTM

In [ ]:
train_lstm(model_lstm,train_dl,30,loss_fn,optimizer,device)

# Check which names are predicted more poorly

In [ ]:
outputs=[]
model.eval()
with torch.no_grad():
      for batch in test_dl:
          val_X,val_y=batch
          val_X=val_X.to(device)
          val_y=val_y.to(device)
          x=model(val_X)
          probs=F.softmax(x,dim=1)
          preds=probs.argmax(dim=1)
          outputs.append(preds)

In [ ]:
preds=torch.cat(outputs)
names=df_reduced.iloc[indices]
names_val=names.iloc[-nb_validation:]
names_val['labels']=y_test
names_val['preds']=preds.cpu().numpy()

In [ ]:
names_val[['category','labels']].drop_duplicates()

,category,labels
759073,Bouwbedrijf,1
1872036,Kapper,0
988954,Computersoftware,2
2473525,Reclame,3
1134912,Diversen,5
10743,Administratiekantoor,4


#### wrongly predicted categories

In [ ]:
error=names_val[names_val['labels']!=names_val['preds']]

In [ ]:
error.head()

,name,category,labels,preds
759073,Quality Finish BV,Bouwbedrijf,1,0
1877486,Poquito by Poquito,Kapper,0,5
2477701,Nadia Attouani,Reclame,3,5
2470565,Corporate Design,Reclame,3,2
1134912,Ciris Capital BV,Diversen,5,1


In [ ]:
error[error['labels']==4][:20]

,name,category,labels,preds
10743,Raede Groep Ten,Administratiekantoor,4,2
50928,Countforyou,Administratiekantoor,4,5
49073,Baete BV,Administratiekantoor,4,1
55328,Koster M de,Administratiekantoor,4,1
51318,Delphi Ita,Administratiekantoor,4,2
56325,Mathot Home Office BV,Administratiekantoor,4,1
48892,Autosloperij Rode Ben BV,Administratiekantoor,4,5
48973,B en A Dienstverlening De Waart,Administratiekantoor,4,1
53009,Gerrit van Eerbeek BV,Administratiekantoor,4,1
57335,ok4backup,Administratiekantoor,4,5
